In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    #model = "mistral-small:22b-instruct-2409-q6_K",
    model = "qwen2.5:32b-instruct-q6_K",
    #model = "finalend/hermes-3-llama-3.1:70b-q3_K_M",
    temperature = 0.7,
    top_p=0.8,
    repeat_penalty = 1.05,
    num_ctx=32678,
    num_predict = 8192,
)

In [2]:
problem = """Five people (A, B, C, D, and E) are in a room. A is watching TV with B, D is sleeping, B is eating chow min, and E is playing table tennis. Suddenly, a call comes on the telephone. B goes out of the room to pick up the call. What is C doing?"""

In [3]:
if False:
    from understanding import ProblemUnderstanding
    from langchain_core.messages import HumanMessage
    from models import State

    import yaml


    initial_state = State(messages=[HumanMessage(content=problem)])

    problem_understanding = ProblemUnderstanding("simple_prompts", llm, None)
    interim = problem_understanding.understand_problem(initial_state, terms_recursion_limit=0, recursion_limit=1)

    print(yaml.dump(interim["understanding"]))

In [3]:
import yaml

from models import Understanding

saved_understanding_yaml = """goals:
- Identify all the activities each person is engaged in before B leaves the room.
- Determine if there are any constraints or implications on C's activity based on
  the given information.
- Figure out what happens to C when B leaves the room, as it might give a clue about
  C's activity.
information:
- Five people (A, B, C, D, and E) are in a room.
- A is watching TV with B.
- D is sleeping.
- B is eating chow mein.
- E is playing table tennis.
- The phone rings and B goes out to answer it.
- The problem does not provide any direct information about what C is doing.
- The central theme revolves around deducing the activity of one person (C) from limited
  information provided about others.
- The author's purpose seems to be testing the reader's ability to discern given details
  and infer missing ones logically.
rephrasal: There are five people (A, B, C, D, E) in a room. A and B are watching TV,
  D is sleeping, B is also eating chow mein, and E is playing table tennis. When the
  phone rings, B leaves to answer it. What activity was C engaged in?
terms:
  doubts:
  - What does 'playing table tennis' mean? Does E need a partner or can they practice
    alone?
  - Does 'watching TV with B' imply A and B are the only ones watching, or could C
    also be watching without it being mentioned?
  - Could D's sleeping be interrupted by the phone call or is it assumed to continue
    unaffected?
  term_definitions: {}"""
saved_understanding:Understanding = yaml.safe_load(saved_understanding_yaml)

In [5]:
if False:
    from langchain_core.messages import AIMessage, HumanMessage

    from devise_plan import DevisePlan
    from models import State


    plan_preparation = DevisePlan("prompts", llm)

    initial_state = State(
        messages=[AIMessage(content=saved_understanding_yaml), HumanMessage(content=problem)],
        understanding=saved_understanding
    )

    result = plan_preparation.devise_plan(state=initial_state, review=False)

    print(yaml.dump(result["plan"]))

In [4]:
from models import Plan

saved_plan_yaml = """plan_for_obstacles: Avoid assuming activities for C beyond what can be inferred from
  the provided information. Ensure that any assumptions about other participants'
  activities, such as E playing table tennis alone or needing a partner, do not override
  the main focus on A and B watching TV together with possible involvement of C. Refrain
  from speculating on D's reaction to the phone call affecting C's activity without
  direct evidence. Maintain focus on context clues that suggest C was engaged in an
  activity with others, particularly those related to the TV-watching scenario.
selected_strategy:
  description: Consider whether A might need someone else (possibly C) to continue
    watching TV as implied by 'watching TV with B', and explore the possibility that
    C was part of this activity before B left.
  evaluation: This strategy is moderately feasible and effective. It uses context
    clues ('watchting TV with B') to reasonably infer C's involvement in a shared
    activity.
strategies:
- description: Assume that since C is not mentioned as being engaged in a specific
    activity, explore whether the absence of information about C implies C could be
    doing anything, including activities not explicitly stated for others.
  evaluation: This strategy is highly feasible but less effective due to its broadness.
    It suggests multiple possibilities without narrowing down based on given context.
- description: Analyze if E playing table tennis requires another person and deduce
    if C was possibly involved with E. Consider implications on C's activity if E
    is self-playing or needs a partner.
  evaluation: This strategy is moderately feasible as it hinges on the assumption
    about whether table tennis can be played alone. It could lead to an effective
    deduction about C's involvement if E requires a partner.
- description: "Determine if the phone call could affect D's sleeping state, leading to a potential inference about C\u2019s role in maintaining quiet conditions for D during B's absence."
  evaluation: This strategy is less feasible as it introduces assumptions not directly
    supported by the given information. The phone call does not necessarily imply
    an impact on D or C's actions.
- description: Consider whether A might need someone else (possibly C) to continue
    watching TV as implied by 'watching TV with B', and explore the possibility that
    C was part of this activity before B left.
  evaluation: This strategy is moderately feasible and effective. It uses context
    clues ('watching TV with B') to reasonably infer C's involvement in a shared activity."""
saved_plan: Plan = yaml.safe_load(saved_plan_yaml)

In [7]:
if False:
    from langchain_core.messages import AIMessage, HumanMessage

    from carry_out_plan import CarryOutPlan
    from models import State


    executor = CarryOutPlan("prompts", llm)


    initial_state: State = State(
        messages=[
            AIMessage(content=saved_understanding_yaml), 
            AIMessage(content=saved_plan_yaml), 
            HumanMessage(content=problem)
        ],
        understanding=saved_understanding,
    )
    initial_state.plan = saved_plan
    #pprint(initial_state.model_dump(), indent=2, width=104)

    result = executor.carry_out_plan(initial_state, adjustment_limit=1)
    print("EXECUTION", yaml.dump(result["execution"]), sep="\n")
    print("PLAN", yaml.dump(result["plan"]), sep="\n")


--- #[internal step] Creating plan...

================================ System Message ================================

You are working on executing a plan to solve a problem using Polya's method. Your tasks involve:
- Determine the step-by-step implementation of the strategy.
- Implement the steps.
- Monitor progress and verify each step.
- Adjust the plan as needed when encountering obstacles.

Use the full context of the conversation in these tasks. Use the central theme and the purpose listed in the information to determine how strictly logical vs how exploratory or creative that work towards the solution should be.

BE SURE TO ESCAPE ANY CHARACTERS AS REQUIRED TO ENSURE VALID JSON.
================================== Ai Message ==================================

goals:
- Identify all the activities each person is engaged in before B leaves the room.
- Determine if there are any constraints or implications on C's activity based on
  the given information.
- Figure out what happen

In [5]:
from models import Execution

saved_execution_yaml = """previous_adjustments:
- description: Consider whether A might need someone else (possibly C) to continue
    watching TV as implied by 'watching TV with B', and explore the possibility that
    C was part of this activity before B left.
  is_adjusted: true
  original: Consider whether A might need someone else (possibly C) to continue watching
    TV as implied by 'watching TV with B', and explore the possibility that C was
    part of this activity before B left.
  previous_progress: The steps have identified all the activities each person is engaged
    in before B leaves the room, but there is no explicit mention of what C is doing.
    The plan has not yet determined any constraints or implications on C's activity
    based on the given information.
  recommendation_from_plan_for_obstacles: Avoid assuming activities for C beyond what
    can be inferred from the provided information. Ensure that any assumptions about
    other participants' activities do not override the main focus on A and B watching
    TV together with possible involvement of C.
result: 'Failed to execute plan after multiple adjustments

  Consider changing the strategy.


  Plan Review:


  {"plan_for_obstacles": "Avoid assuming activities for C beyond what can be inferred
  from the provided information. Ensure that any assumptions about other participants''
  activities, such as E playing table tennis alone or needing a partner, do not override
  the main focus on A and B watching TV together with possible involvement of C. Refrain
  from speculating on D''s reaction to the phone call affecting C''s activity without
  direct evidence. Maintain focus on context clues that suggest C was engaged in an
  activity with others, particularly those related to the TV-watching scenario.",
  "selected_strategy": {"description": "Consider whether A might need someone else
  (possibly C) to continue watching TV as implied by ''watching TV with B'', and explore
  the possibility that C was part of this activity before B left.", "is_adjusted":
  true, "original": "Consider whether A might need someone else (possibly C) to continue
  watching TV as implied by ''watching TV with B'', and explore the possibility that
  C was part of this activity before B left.", "previous_progress": "The steps have
  identified all the activities each person is engaged in before B leaves the room,
  but there is no explicit mention of what C is doing. The plan has not yet determined
  any constraints or implications on C''s activity based on the given information.",
  "recommendation_from_plan_for_obstacles": "Avoid assuming activities for C beyond
  what can be inferred from the provided information. Ensure that any assumptions
  about other participants'' activities do not override the main focus on A and B
  watching TV together with possible involvement of C."}, "strategies": [{"description":
  "Assume that since C is not mentioned as being engaged in a specific activity, explore
  whether the absence of information about C implies C could be doing anything, including
  activities not explicitly stated for others.", "evaluation": "This strategy is highly
  feasible but less effective due to its broadness. It suggests multiple possibilities
  without narrowing down based on given context."}, {"description": "Analyze if E
  playing table tennis requires another person and deduce if C was possibly involved
  with E. Consider implications on C''s activity if E is self-playing or needs a partner.",
  "evaluation": "This strategy is moderately feasible as it hinges on the assumption
  about whether table tennis can be played alone. It could lead to an effective deduction
  about C''s involvement if E requires a partner."}, {"description": "Determine if
  the phone call could affect D''s sleeping state, leading to a potential inference
  about C\u2019s role in maintaining quiet conditions for D during B''s absence.",
  "evaluation": "This strategy is less feasible as it introduces assumptions not directly
  supported by the given information. The phone call does not necessarily imply an
  impact on D or C''s actions."}, {"description": "Consider whether A might need someone
  else (possibly C) to continue watching TV as implied by ''watching TV with B'',
  and explore the possibility that C was part of this activity before B left.", "evaluation":
  "This strategy is moderately feasible and effective. It uses context clues (''watching
  TV with B'') to reasonably infer C''s involvement in a shared activity."}]}'
should_change_strategy: true
steps:
- action: Identify all the activities each person is engaged in before B leaves the
    room.
  is_verified: true
  result: A is watching TV with B, D is sleeping, B is eating chow mein, and E is
    playing table tennis. There is no explicit mention of C's activity.
- action: Determine if there are any constraints or implications on C's activity based
    on the given information.
  is_verified: false
  result: There are no explicit constraints or implications provided about C's activity
    in the given information. The activities of A, B, D, and E do not directly inform
    us about what C is doing. Specifically, 'watching TV with B' does not necessarily
    exclude C from watching TV as well, but it also does not confirm it. Similarly,
    there is no indication that C needs to be involved in maintaining quiet conditions
    for D's sleep or any other activities of the others. The information given does
    not specify a direct role or constraint for C.
- action: Assess whether 'watching TV with B' implies that A and B are the only ones
    watching, or if C could also be watching without it being explicitly mentioned.
- action: Explore the possibility that C was involved in the TV-watching activity
    before B left to answer the phone."""
saved_execution: Execution = yaml.safe_load(saved_execution_yaml)

from models import Plan

saved_plan_yaml = """plan_for_obstacles: Avoid assuming activities for C beyond what can be inferred from
  the provided information. Ensure that any assumptions about other participants'
  activities, such as E playing table tennis alone or needing a partner, do not override
  the main focus on A and B watching TV together with possible involvement of C. Refrain
  from speculating on D's reaction to the phone call affecting C's activity without
  direct evidence. Maintain focus on context clues that suggest C was engaged in an
  activity with others, particularly those related to the TV-watching scenario.
selected_strategy:
  description: Consider whether A might need someone else (possibly C) to continue
    watching TV as implied by 'watching TV with B', and explore the possibility that
    C was part of this activity before B left.
  is_adjusted: true
  original: Consider whether A might need someone else (possibly C) to continue watching
    TV as implied by 'watching TV with B', and explore the possibility that C was
    part of this activity before B left.
  previous_progress: The steps have identified all the activities each person is engaged
    in before B leaves the room, but there is no explicit mention of what C is doing.
    The plan has not yet determined any constraints or implications on C's activity
    based on the given information.
  recommendation_from_plan_for_obstacles: Avoid assuming activities for C beyond what
    can be inferred from the provided information. Ensure that any assumptions about
    other participants' activities do not override the main focus on A and B watching
    TV together with possible involvement of C.
strategies:
- description: Assume that since C is not mentioned as being engaged in a specific
    activity, explore whether the absence of information about C implies C could be
    doing anything, including activities not explicitly stated for others.
  evaluation: This strategy is highly feasible but less effective due to its broadness.
    It suggests multiple possibilities without narrowing down based on given context.
- description: Analyze if E playing table tennis requires another person and deduce
    if C was possibly involved with E. Consider implications on C's activity if E
    is self-playing or needs a partner.
  evaluation: This strategy is moderately feasible as it hinges on the assumption
    about whether table tennis can be played alone. It could lead to an effective
    deduction about C's involvement if E requires a partner.
- description: "Determine if the phone call could affect D's sleeping state, leading to a potential inference about C\u2019s role in maintaining quiet conditions for D during B's absence."
  evaluation: This strategy is less feasible as it introduces assumptions not directly
    supported by the given information. The phone call does not necessarily imply
    an impact on D or C's actions.
- description: Consider whether A might need someone else (possibly C) to continue
    watching TV as implied by 'watching TV with B', and explore the possibility that
    C was part of this activity before B left.
  evaluation: This strategy is moderately feasible and effective. It uses context
    clues ('watching TV with B') to reasonably infer C's involvement in a shared activity."""
saved_plan: Plan = yaml.safe_load(saved_plan_yaml)


In [6]:
# Set up initial state.
from langchain_core.messages import AIMessage, HumanMessage
from models import Reflections, State

initial_state: State = State(
    messages=[
        AIMessage(content=saved_understanding_yaml),
        AIMessage(content=saved_plan_yaml),
        AIMessage(content=saved_execution_yaml),
        HumanMessage(content=problem)
    ],
    understanding=saved_understanding,
    plan=saved_plan,
    execution=saved_execution,
)

# Initialize the Reflection node.
from reflection import Reflection

reflector = Reflection("prompts", llm)

# Perform reflection.
result: Reflections = reflector.reflect_on_solution(initial_state)

print(yaml.dump(result))


DESCRIPTION
Consider whether A might need someone else (possibly C) to continue watching TV as implied by 'watching TV with B', and explore the possibility that C was part of this activity before B left.
PLAN
{'description': "Consider whether A might need someone else (possibly C) to continue watching TV as implied by 'watching TV with B', and explore the possibility that C was part of this activity before B left."}
bingo
Failed to execute plan after multiple adjustments
Consider changing the strategy.

Plan Review:

{"plan_for_obstacles": "Avoid assuming activities for C beyond what can be inferred from the provided information. Ensure that any assumptions about other participants' activities, such as E playing table tennis alone or needing a partner, do not override the main focus on A and B watching TV together with possible involvement of C. Refrain from speculating on D's reaction to the phone call affecting C's activity without direct evidence. Maintain focus on context clues tha